In [152]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from  more_itertools import unique_everseen
import itertools
from datetime import datetime as dt
from urllib.request import *
import re
import operator

In [49]:
pd.options.mode.chained_assignment = None

In [50]:
headlinelinks=[]
headlinetitles=[]
timestamps=[]

url = ["https://www.reuters.com/news/archive/mergersNews?view=page&page=1&pageSize=10","https://www.reuters.com/news/archive/EUmergersNews?view=page&page=1&pageSize=10"]

for i in range(20):
    new_page = "page="+str(i)
    for j in url:
        new_url = j
        new_url = new_url.replace("page=1",new_page)
        html = urlopen(new_url)
        soup = bs(html,'html.parser')
        headlines = soup.find_all('div',class_='story-content')
        headlinelinks.extend( [x.find('a')['href'] for x in headlines] )
        headlinetitles.extend( [x.find('h3').text for x in headlines] )
        datetm = [x.find('span',class_='timestamp').text for x in headlines]
        datetm = [re.sub(' EDT','',x) for x in datetm]
        timestamps.extend(datetm)

In [51]:
removelist=['$','million','mln','']
headlinetitles = [x.replace("\t","") for x in headlinetitles]
headlinetitles = [x.replace("\n","") for x in headlinetitles]
#headlinetitles = list(unique_everseen(headlinetitles))
cleanedheadlinetitles = [re.sub("'s","",x) for x in headlinetitles]
cleanedheadlinetitles = [re.sub('[^a-zA-Z|\s]',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(million|mln|billion|bln)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(a|to|be)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s+',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('BRIEF','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('UPDATE','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [x.strip() for x in cleanedheadlinetitles]

In [52]:
newtimestamps = []
for item in timestamps:
    try:
        newtimestamps.append(dt.strptime(item,'%b %d %Y'))
    except:
        item = re.sub(r'(am|pm)',r' \1',item)
        currentime = dt.strptime(item,'%I:%M %p')
        currentdate = dt.today()
        currentdate = currentdate.replace(hour = currentime.hour, minute = currentime.minute)
        newtimestamps.append(currentdate)

In [53]:
headlinesframe=[]
headlinesframe = pd.DataFrame(cleanedheadlinetitles,columns=['titles'])
headlinesframe['link'] = pd.DataFrame(headlinelinks)
headlinesframe['datetime'] = pd.DataFrame(newtimestamps)
headlinesframe.sort_values('datetime',ascending=False,inplace=True)
headlinesframededup = headlinesframe.drop_duplicates(subset=['titles'], keep='first')

In [54]:
namelist = pd.read_excel('NameList.xlsx','ModNamesList')
namelistmod = namelist.iloc[:,1:]
searcharray = namelistmod.as_matrix()
newsearcharray=[]
for row in searcharray:
    newrow = [x.strip() for x in row if not pd.isnull(x)]
    newsearcharray.append(newrow)

In [133]:
def checknames(headlines):
    rowname = []
    names = []

    for new in headlines:
        words = new.split()
        s = " "
        newwords=[]    
        for i in range(int(len(words))):
            for j in range(1,5):
                newwords.append(s.join(words[i:i+j])) 
        newwords = list(sorted(set(newwords),key=lambda x:newwords.index(x)))
        newwords2 = sorted(newwords,key = lambda x:(len(x.split()),len(newwords)-newwords.index(x)),reverse=True)
    
        cos = []
        foundwords=[]
    
        for word in newwords2:
        
            founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]          
            foundd = [rowd for ix,rowd in enumerate(foundwords) if rowd.strip().lower() == word.lower()]
        
            if len(foundd)!=0:
                continue
        
            if len(founds)!=0:
                cos.extend(founds)
                neww=[]
                w = word.split()
                for i in range(int(len(w))):
                    for j in range(1,5):
                        neww.append(s.join(w[i:i+j])) 
                neww = list(set(neww))
                foundwords.extend(neww)
                continue
        
        cos = list(sorted(set(cos),key=lambda x:cos.index(x)))
        names.append([newsearcharray[i][0] for i in cos])
    
    return(names)

In [155]:
def check_mna(linknames):
    relevant = []
    wordlist = ["buy","bought","buying","acquire","acquired","acquires","acquisition","merge","merger","merged","spinoff","buyout","lbo","stake","sale","divest","divesture"]
    for link in linknames:
        founds = [ix for ix, row in enumerate(wordlist) if row in link.lower()]
        if len(founds)!=0:
            relevant.append(True)
        else:
            relevant.append(False)
    return(relevant)

In [135]:
companeslist = checknames(headlinesframededup['titles'])
relevantlist = check_mna(headlinesframededup['titles'])
headlinesframededup['companies']=companeslist  
headlinesframededup['relevant']=relevantlist
headlinesframededup['refdCount'] = [len(x) for x in headlinesframededup['companies']]

In [136]:
headlinesframerel = headlinesframededup.copy()
headlinesframerel = headlinesframerel[headlinesframerel.relevant==True]
headlinesframerel = headlinesframerel[headlinesframerel.refdCount>0]

In [157]:
headlinesframerel

,titles,link,datetime,companies,relevant,refdCount
32,Cisco buys BroadSoft for in software push,/article/broadsoft-ma-cisco-systems/update-3-c...,2017-10-23 11:34:51.500099,"[CISCO SYSTEMS INC, BROADSOFT INC]",True,2
33,Smith Nephew buy Rotation Medical for up mln,/article/rotationmedical-ma-smith-nephew/updat...,2017-10-23 10:15:51.500132,[SMITH A O CORP],True,1
27,DIGI INTERNATIONAL ACQUIRES TEMPALERT,/article/brief-digi-international-acquires-tem...,2017-10-23 09:58:51.499933,[DIGI INTL INC],True,1
41,Apple Hospitality REIT acquires two Residence ...,/article/brief-apple-hospitality-reit-acquires...,2017-10-23 09:18:51.500410,"[APPLE HOSPITALITY REIT, MARRIOTT INTL INC, MA...",True,3
44,Cisco announces agreement acquire Broadsoft,/article/brief-cisco-announces-agreement-to-ac...,2017-10-23 08:28:51.500578,"[CISCO SYSTEMS INC, BROADSOFT INC]",True,2
60,Cisco buy BroadSoft in deal,/article/broadsoft-ma-cisco-systems/cisco-to-b...,2017-10-23 08:06:51.500987,"[CISCO SYSTEMS INC, BROADSOFT INC]",True,2
63,CORRECTED Cisco nears deal acquire BroadSoft s...,/article/broadsoft-ma-cisco-systems/corrected-...,2017-10-23 07:55:51.501089,"[CISCO SYSTEMS INC, BROADSOFT INC]",True,2
64,Potlatch buy rival Deltic Timber for stock com...,/article/deltictimber-ma-potlatch/update-1-pot...,2017-10-23 07:53:51.501122,"[DELTIC TIMBER CORP, POTLATCH CORP, LUMBER LIQ...",True,3
66,Potlatch buy Deltic Timber in all stock deal,/article/deltictimber-ma-potlatch/potlatch-to-...,2017-10-23 07:16:51.501189,"[DELTIC TIMBER CORP, POTLATCH CORP]",True,2
81,The Hartford acquire Aetna U S group life and ...,/article/brief-the-hartford-to-acquire-aetnas-...,2017-10-23 06:58:51.501596,"[HARTFORD FINL SVCS GRP, AETNA INC]",True,2


In [132]:
### for debug
news = ["Cisco announces agreement to acquire Broadsoft"]
names=[]

for new in news:
    words = new.split()
    s = " "
    newwords=[]    
    for i in range(int(len(words))):
        for j in range(1,6):
            newwords.append(s.join(words[i:i+j])) 
    newwords = list(sorted(set(newwords),key=lambda x:newwords.index(x)))
    newwords2 = sorted(newwords,key = lambda x:(len(x.split()),len(newwords)-newwords.index(x)),reverse=True)
    
    cos = []
    foundwords=[]
    
    
    for word in newwords2:
        
        founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]          
        foundd = [rowd for ix,rowd in enumerate(foundwords) if rowd.strip().lower() == word.lower()]
        
        if len(foundd)!=0:
            continue
        
        if len(founds)!=0:
            cos.extend(founds)
            neww=[]
            w = word.split()
            for i in range(int(len(w))):
                for j in range(1,5):
                    neww.append(s.join(w[i:i+j])) 
            neww = list(set(neww))
            foundwords.extend(neww)
            continue
        
    cos = list(sorted(set(cos),key=lambda x:cos.index(x)))
    names.append([newsearcharray[i][0] for i in cos])
names


[['CISCO SYSTEMS INC', 'BROADSOFT INC']]

In [12]:
#save output
writer = pd.ExcelWriter('output4.xlsx')
headlinesframededup.to_excel(writer)



    Only loading the 'en' tokenizer.

